# Test for å lage kontroller av Finansiering av investeringene

## Krav for at tabellen skal være korrekt
1. Andel av brutto investeringsutgifter skal være 100 % per kommune per årgang
2. Beløp summert per kommune per årgang til tilsvare Brutto investeringsutgifter
3. Årlig utvikling skal være fornuftig 
    - Definisjon av fornuftig utvikling må bestemmes

### Steg
- Importerer tabeller fra Linux. Trenger to tabeller:
    - Finansiering av investeringene
    - Finansielle grunnlagsdata 
    
1. Kontroller om andel er 100 %
    - Ok med avvik på 1 %
    - Hvor mange kommuner har høyere avvik
        - Er dette mer/mindre enn tidligere årganger
2. Kontroller om summer av investeringene tilsvarer brutto investeringsutgifter for kommunen
    - Hvor mange avviker
    - Hvor betydelig er avviket
    - Er det sammenheng mellom kommunene som avviker på kontroll 1 og 2
3. Kontroller den årlige utviklingen for alle tilgjengelige årganger
    - Prosentvis utvikling 
    - Plotte utviklingen
    - Se om HP-metoden fra Kostra biblioteket kan være et alternativ



In [ ]:
import pandas as pd
import numpy as np
import glob

In [ ]:
#Sjekker hva som ligger i mappen på linux
#Har tidligere lagret test-data på egen Linux mappe
paths = glob.glob("/ssb/bruker/mtk/Testdata/*")
paths

In [ ]:
#Henter filen, sjekker innhold
fininv = pd.read_csv("/ssb/bruker/mtk/Testdata/finansiering_av_investeringene.csv", sep = ';')
fininv.info()
fininv.head(5)

In [ ]:
fingr = pd.read_csv('/ssb/bruker/mtk/Testdata/finansielle_grunnlagsdata.csv', sep = ';')
fingr.dtypes
fingr.info()
fingr.head()

In [ ]:
#Retter datatyde
#fininv['andelinv'] = fininv['andelinv'].astype("float64")
#fininv.convert_dtypes().info()

In [ ]:
#Filtrerer ut Halden og AGI23
AGI23 = fininv[((fininv['kommuneregion'] == '3001')  | (fininv['kommuneregion'] == '0101')) & (fininv['art_kap'] == 'AGI23')]
AGI23.head(10)

In [ ]:
#Grupperer finansiering av investeringene etter kommuneregion og periode
gruppert_alle = fininv.groupby(['periode','kommuneregion'])[['belop','andelinv']].sum()
gruppert_alle

In [ ]:
#Filtrerer ut observasjoner hvor andelinv avviker fra 100 med intervall på 0.5
diff = gruppert_alle[(gruppert_alle['andelinv'] >= 100.5) | (gruppert_alle['andelinv'] <= 99.5)]
diff

In [ ]:
#Filtrerer ut 2021
p2021 = fininv[(fininv['periode'] == 2021)]
p2021

#Grupperer finansiering av investeringene etter kommuneregion og periode
gruppert_2021 = p2021.groupby(['periode','kommuneregion'])[['belop','andelinv']].sum()
gruppert_2021

#Filtrerer ut observasjoner hvor andelinv avviker fra 100 med intervall på 1.0
diff2021 = gruppert_2021[(gruppert_2021['andelinv'] >= 101.0) | (gruppert_2021['andelinv'] <= 99.0)]
diff2021


In [ ]:
len(diff2021)